In [57]:
import pandas as pd

In [58]:
df=pd.read_csv('../data/output/categorical_dfs/testing_sample_cleaned_categorical.csv')
columns_final_df = pd.read_csv('../data/output/categorical_dfs/columns_final_woe.csv')
df.head() 

,MOB,ID,customer_id,Var1,Var4,Var5,Var6,Var7,Var15,Var16,...,Var25_q,Var26_q,Var20_1,Var21_1,Var22_1,Var23_1,Var17_1,working_months_1,ii_ratio,idi_ratio
0,9,36034977,32653719,1,1,1,1,1,1,1,...,2.0,3.0,1,3,3,3,1,2,5,6
1,9,36034978,32832365,2,1,2,6,8,0,0,...,4.0,1.0,1,1,1,1,4,5,9,9
2,9,36034979,32544742,1,0,2,1,0,0,0,...,1.0,3.0,0,0,0,0,0,4,4,2
3,9,36034981,32592943,2,2,8,3,2,0,2,...,0.0,2.0,2,2,2,2,3,6,4,0
4,9,36034982,32601182,2,4,6,1,2,0,0,...,1.0,10.0,0,0,0,0,2,7,3,7


In [59]:
# calculate WOE for each category
def calculate_woe(df, feature, target):

    df = df[[feature, target]]
    df['non_target'] = 1 - df[target]

    total_non_target = df['non_target'].sum()
    total_target = df[target].sum()

    df = df.groupby(feature).agg({target: 'sum', 'non_target': 'sum'}).reset_index()

    df['total_target'] = total_target
    df['total_non_target'] = total_non_target

    df['woe'] = (df['non_target'] / total_non_target) / (df[target] / total_target)
    df['iv'] = (df['non_target'] / total_non_target - df[target] / total_target) * df['woe']

    df.rename(columns = {feature:'category'}, inplace = True)
    df['variable'] = feature

    # Move the variable column to the front
    cols = list(df.columns)
    cols = [cols[-1]] + cols[:-1]
    df = df[cols]

    return df

In [60]:
columns_to_calc_woe = df.columns
columns_to_calc_woe = columns_to_calc_woe.drop(['target', 'ID', 'customer_id'])

In [61]:
columns_to_calc_woe

Index(['MOB', 'Var1', 'Var4', 'Var5', 'Var6', 'Var7', 'Var15', 'Var16',
       'Var24', 'Var29', 'Var30', 'Var3', 'Var11', 'Var12', 'Var14', 'Var27',
       'Var28', 'income', 'loan_desc', 'Var8_q', 'Var25_q', 'Var26_q',
       'Var20_1', 'Var21_1', 'Var22_1', 'Var23_1', 'Var17_1',
       'working_months_1', 'ii_ratio', 'idi_ratio'],
      dtype='object')

In [62]:
# define dataframe to store WOE values
df_woe_all = pd.DataFrame()

for col in columns_to_calc_woe:
    df_woe = calculate_woe(df, col, 'target')
    df_woe_all = pd.concat([df_woe_all, df_woe], axis=0)

/var/folders/rh/w6cl6rd95vj2g4s03scz_4fc0000gn/T/ipykernel_34639/2911622210.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['non_target'] = 1 - df[target]
/var/folders/rh/w6cl6rd95vj2g4s03scz_4fc0000gn/T/ipykernel_34639/2911622210.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['non_target'] = 1 - df[target]
/var/folders/rh/w6cl6rd95vj2g4s03scz_4fc0000gn/T/ipykernel_34639/2911622210.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

In [63]:
df_woe_all

,variable,category,target,non_target,total_target,total_non_target,woe,iv
0,MOB,0.0,6.0,369.0,119.0,3427.0,2.135541,0.122269
1,MOB,1.0,7.0,332.0,119.0,3427.0,1.646922,0.062672
2,MOB,2.0,21.0,363.0,119.0,3427.0,0.600233,-0.042345
3,MOB,3.0,12.0,328.0,119.0,3427.0,0.949129,-0.004869
4,MOB,4.0,15.0,332.0,119.0,3427.0,0.768563,-0.022421
...,...,...,...,...,...,...,...,...
5,idi_ratio,5.0,8.0,347.0,119.0,3427.0,1.506164,0.051252
6,idi_ratio,6.0,13.0,341.0,119.0,3427.0,0.910844,-0.008871
7,idi_ratio,7.0,9.0,346.0,119.0,3427.0,1.334954,0.033818
8,idi_ratio,8.0,9.0,345.0,119.0,3427.0,1.331096,0.033332


In [64]:
# sum IV grouped by variable
df_iv = df_woe_all.groupby('variable').agg({'iv': 'sum'}).reset_index()

In [65]:
# order by IV
df_iv = df_iv.sort_values('iv', ascending=False)
df_iv

,variable,iv
24,Var8_q,0.246749
14,Var26_q,0.188713
13,Var25_q,0.162495
0,MOB,0.158203
25,idi_ratio,0.154490
29,working_months_1,0.153527
19,Var30,0.140977
27,income,0.125902
4,Var14,0.115494
23,Var7,0.109336


In [66]:
# df_iv.to_csv('../data/output/WOE_variables_1.csv', index = False)

In [67]:
df.shape[0]

3546

In [68]:
# Add new column with WOE values to df from df_woe_all

for col in columns_to_calc_woe:
    df_woe2 = df_woe_all[df_woe_all['variable'] == col]
    df_woe2 = df_woe2[['category', 'woe']]
    df = df.merge(df_woe2, how='left', left_on=col, right_on='category')
    df = df.rename(columns = {'woe': col + '_woe'})
    df = df.drop(columns=['category'])

In [69]:
df

,MOB,ID,customer_id,Var1,Var4,Var5,Var6,Var7,Var15,Var16,...,Var25_q_woe,Var26_q_woe,Var20_1_woe,Var21_1_woe,Var22_1_woe,Var23_1_woe,Var17_1_woe,working_months_1_woe,ii_ratio_woe,idi_ratio_woe
0,9,36034977,32653719,1,1,1,1,1,1,1,...,1.061019,0.650290,0.764672,0.798658,1.064877,0.931767,0.845783,0.825941,1.197987,0.910844
1,9,36034978,32832365,2,1,2,6,8,0,0,...,0.951444,1.840385,0.764672,0.847394,0.877214,1.024038,1.194514,1.751094,0.735720,0.845783
2,9,36034979,32544742,1,0,2,1,0,0,0,...,0.787083,0.650290,1.138751,1.241014,1.336357,1.310173,1.197987,1.350387,0.648186,1.501824
3,9,36034981,32592943,2,2,8,3,2,0,2,...,0.669682,0.590312,2.482784,0.858185,0.827068,0.737380,1.197987,0.897488,0.648186,0.439395
4,9,36034982,32601182,2,4,6,1,2,0,0,...,0.787083,1.008513,1.138751,1.241014,1.336357,1.310173,1.082765,0.787083,1.506164,1.334954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3541,0,36039969,32399390,1,8,1,1,8,3,5,...,1.061019,1.849066,1.138751,1.241014,1.336357,1.310173,0.787083,0.897488,1.334954,1.334954
3542,0,36039970,32880524,1,7,3,1,7,0,0,...,0.910844,1.008513,1.138751,0.858185,0.827068,0.737380,0.843303,0.787083,1.197987,1.331096
3543,0,36039972,32807051,1,8,5,1,6,0,2,...,1.937613,1.215349,0.764672,0.847394,0.877214,1.024038,1.194514,0.510295,1.334954,0.910844
3544,0,36039973,32826431,2,9,6,1,7,0,0,...,0.951444,1.215349,1.138751,1.241014,1.336357,1.310173,0.787083,0.992535,1.194514,0.910844


In [70]:
# select all columns that are not in columns_to_calc_woe
columns_to_keep = df.columns.difference(columns_to_calc_woe)
columns_to_keep
columns_to_keep = columns_to_keep.drop(['ID', 'customer_id'])

In [71]:
df_woe = df[columns_to_keep]
df_woe

,MOB_woe,Var11_woe,Var12_woe,Var14_woe,Var15_woe,Var16_woe,Var17_1_woe,Var1_woe,Var20_1_woe,Var21_1_woe,...,Var5_woe,Var6_woe,Var7_woe,Var8_q_woe,idi_ratio_woe,ii_ratio_woe,income_woe,loan_desc_woe,target,working_months_1_woe
0,1.641961,3.507149,1.034942,0.932271,1.155092,1.277356,0.845783,1.034942,0.764672,0.798658,...,0.798658,1.034179,0.848264,0.915852,0.910844,1.197987,0.550853,0.865435,0.0,0.825941
1,1.641961,1.013005,inf,0.586592,0.933214,0.898490,1.194514,1.031310,0.764672,0.847394,...,1.059090,0.920193,0.989641,0.915852,0.845783,0.735720,0.885468,1.229554,0.0,1.751094
2,1.641961,1.013005,1.034942,0.932271,0.933214,0.898490,1.197987,1.034942,1.138751,1.241014,...,1.059090,1.034179,0.845783,0.915852,1.501824,0.648186,0.550853,0.865435,0.0,1.350387
3,1.641961,1.857747,0.878674,1.642457,0.933214,0.946236,1.197987,1.031310,2.482784,0.858185,...,1.253545,0.873067,0.843303,0.668442,0.439395,0.648186,0.926870,1.009484,0.0,0.897488
4,1.641961,1.013005,0.878674,0.932271,0.933214,0.898490,1.082765,1.031310,1.138751,1.241014,...,1.121593,1.034179,0.843303,0.668442,1.334954,1.506164,0.926870,1.009484,0.0,0.787083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3541,2.135541,1.013005,1.034942,0.586592,1.128538,0.607674,0.787083,1.034942,1.138751,1.241014,...,0.798658,1.034179,0.989641,2.743216,1.334954,1.334954,1.304088,1.179572,0.0,0.897488
3542,2.135541,1.207632,1.034942,0.586592,0.933214,0.898490,0.843303,1.034942,1.138751,0.858185,...,1.011344,1.034179,1.085922,0.915852,1.331096,1.197987,1.304088,1.229554,0.0,0.787083
3543,2.135541,1.013005,1.034942,0.932271,0.933214,0.946236,1.194514,1.034942,0.764672,0.847394,...,1.185585,1.034179,1.194514,0.915852,0.910844,1.334954,0.885468,1.229554,0.0,0.510295
3544,2.135541,1.013005,0.878674,0.932271,0.933214,0.898490,0.787083,1.031310,1.138751,1.241014,...,1.121593,1.034179,1.085922,2.743216,0.910844,1.194514,2.002432,1.179572,0.0,0.992535


In [73]:
# change columns_final_df to list
columns_final_df = columns_final_df['0'].tolist()
columns_final_df

['target',
 'MOB_woe',
 'Var11_woe',
 'Var12_woe',
 'Var14_woe',
 'Var16_woe',
 'Var17_1_woe',
 'Var22_1_woe',
 'Var24_woe',
 'Var25_q_woe',
 'Var26_q_woe',
 'Var27_woe',
 'Var28_woe',
 'Var29_woe',
 'Var30_woe',
 'Var3_woe',
 'Var4_woe',
 'Var5_woe',
 'Var6_woe',
 'Var7_woe',
 'Var8_q_woe',
 'idi_ratio_woe',
 'ii_ratio_woe',
 'income_woe',
 'loan_desc_woe',
 'working_months_1_woe']

In [74]:
df_woe = df_woe[columns_final_df]
df_woe

,target,MOB_woe,Var11_woe,Var12_woe,Var14_woe,Var16_woe,Var17_1_woe,Var22_1_woe,Var24_woe,Var25_q_woe,...,Var4_woe,Var5_woe,Var6_woe,Var7_woe,Var8_q_woe,idi_ratio_woe,ii_ratio_woe,income_woe,loan_desc_woe,working_months_1_woe
0,0.0,1.641961,3.507149,1.034942,0.932271,1.277356,0.845783,1.064877,1.020562,1.061019,...,0.731379,0.798658,1.034179,0.848264,0.915852,0.910844,1.197987,0.550853,0.865435,0.825941
1,0.0,1.641961,1.013005,inf,0.586592,0.898490,1.194514,0.877214,1.020562,0.951444,...,0.731379,1.059090,0.920193,0.989641,0.915852,0.845783,0.735720,0.885468,1.229554,1.751094
2,0.0,1.641961,1.013005,1.034942,0.932271,0.898490,1.197987,1.336357,1.020562,0.787083,...,1.021472,1.059090,1.034179,0.845783,0.915852,1.501824,0.648186,0.550853,0.865435,1.350387
3,0.0,1.641961,1.857747,0.878674,1.642457,0.946236,1.197987,0.827068,1.020562,0.669682,...,1.184097,1.253545,0.873067,0.843303,0.668442,0.439395,0.648186,0.926870,1.009484,0.897488
4,0.0,1.641961,1.013005,0.878674,0.932271,0.898490,1.082765,1.336357,1.020562,0.787083,...,1.288655,1.121593,1.034179,0.843303,0.668442,1.334954,1.506164,0.926870,1.009484,0.787083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3541,0.0,2.135541,1.013005,1.034942,0.586592,0.607674,0.787083,1.336357,1.020562,1.061019,...,0.983854,0.798658,1.034179,0.989641,2.743216,1.334954,1.334954,1.304088,1.179572,0.897488
3542,0.0,2.135541,1.207632,1.034942,0.586592,0.898490,0.843303,0.827068,1.020562,0.910844,...,1.323380,1.011344,1.034179,1.085922,0.915852,1.331096,1.197987,1.304088,1.229554,0.787083
3543,0.0,2.135541,1.013005,1.034942,0.932271,0.946236,1.194514,0.877214,1.020562,1.937613,...,0.983854,1.185585,1.034179,1.194514,0.915852,0.910844,1.334954,0.885468,1.229554,0.510295
3544,0.0,2.135541,1.013005,0.878674,0.932271,0.898490,0.787083,1.336357,0.845783,0.951444,...,1.506164,1.121593,1.034179,1.085922,2.743216,0.910844,1.194514,2.002432,1.179572,0.992535


In [75]:
# save final df with WOE values
df_woe[columns_final_df].to_csv('../data/output/categorical_dfs/testing_sample_woe.csv', index = False)